# MWRpy processing example

#### This minimal working example is based on the Cloudnet API client to fetch raw data, retrieval files and site meta data.
#### See the MWRpy GitHub page for more information: https://github.com/actris-cloudnet/mwrpy

In [3]:
from cloudnet_api_client import APIClient
import matplotlib.pyplot as plt
import requests

# Download raw data (binary files)

In [4]:
data_path = "data/"
site = "munich"
instrument_pid = "https://hdl.handle.net/21.12132/3.4025c06be1ab4983" # check https://cloudnet.fmi.fi/instruments to find the PID of your instrument
date = "2025-08-27"

In [ ]:
client = APIClient()
instruments = client.instruments()
instrument_type = [i.instrument_id for i in instruments if i.pid == instrument_pid][0]
files = client.raw_files(site_id=site, instrument_id=instrument_type, date=date)
binary_files = [f for f in files]
print(len(binary_files))

In [ ]:
binary_filepaths = await client.adownload(binary_files, data_path)

# Download retrieval files

In [ ]:
calibration = client.calibration(instrument_pid, date)
retrieval = calibration["data"]

retrieval_files = []
for file in retrieval["coefficientLinks"]:
    filename = data_path + file.split("/")[-1]
    response = requests.get(file)
    with open(filename, "wb") as f:
        f.write(response.content)
        retrieval_files.append(str(filename))
print(retrieval_files)

# Process and plot Level 1 data

In [ ]:
from mwrpy.level1.write_lev1_nc import lev1_to_nc
site_info = client.site(site_id=site)
site_meta = {
    "name": site_info.id,
    "altitude": site_info.altitude,
    "latitude": site_info.latitude,
    "longitude": site_info.longitude,
}
mwr_raw = lev1_to_nc(
    "1C01",
    data_path,
    instrument_type=instrument_type,
    output_file="mwr_1c.nc",
    coeff_files=retrieval_files,
    instrument_config=site_meta,
)

In [ ]:
from mwrpy.plots.generate_plots import generate_figure
fig_name = generate_figure('mwr_1c.nc', ['tb'], show=True, instrument_type=instrument_type)

# Process and plot Level 2 data (single pointing)

In [ ]:
from mwrpy.level2.lev2_collocated import generate_lev2_single
mwr_prod = generate_lev2_single(None, "mwr_1c.nc", "mwr-single.nc", coeff_files=retrieval_files)

In [ ]:
from mwrpy.plots.generate_plots import generate_figure
fig_name = generate_figure('mwr-single.nc', ['iwv'], show=True, instrument_type=instrument_type)

# Process and plot Level 2 data (multiple pointing)

In [ ]:
from mwrpy.level2.lev2_collocated import generate_lev2_multi
mwr_prod = generate_lev2_multi(site, "mwr_1c.nc", "mwr-multi.nc", coeff_files=retrieval_files)

In [ ]:
from mwrpy.plots.generate_plots import generate_figure
fig_name = generate_figure('mwr-multi.nc', ['temperature'], show=True, instrument_type=instrument_type)